In [ ]:
def load_model(
    model_name: str = "meta-llama/Llama-2-7b-chat-hf", load_on_gpu: bool = True
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    if load_on_gpu:
        config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        model = AutoModelForCausalLM.from_pretrained(
            model_name, low_cpu_mem_usage=True, quantization_config=config, attn_implementation="flash_attention_2"
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id

    return model, tokenizer

# Load the model & tokenizer
model, tokenizer = load_model()

In [ ]:
def tokenize_queries(tokenizer: AutoTokenizer, query: str, max_length: int = 1024) -> torch.Tensor:
    query = [
        {"role": "system", "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: '{Letter}.{Answer}'"},
        {"role": "user", "content": query},
    ]

    input_ids = tokenizer.apply_chat_template(
        query,
        padding='max_length',
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return input_ids

def tokenize_responses(tokenizer: AutoTokenizer, query: str, response: str, max_length: int = 1024) -> torch.Tensor:
    response = [
        {"role": "system", "content": "You answer multiple choice questions with the correct letter answer. Your answer should be in this format: '{Letter}.{Answer}'"},
        {"role": "user", "content": query},
        {"role": "assistant", "content": response},
    ]

    labels = tokenizer.apply_chat_template(
        response,
        padding='max_length',
        max_length=max_length,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    return labels

# Load in the data
train_data = pd.read_csv("data/train.csv")
eval_data = pd.read_csv("data/eval.csv")

# Tokenize the data
train_data["input_ids"] = train_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
train_data["labels"] = train_data.apply(lambda row: tokenize_responses(tokenizer, row["query"], row["response"]), axis=1)

eval_data["input_ids"] = eval_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)
eval_data["labels"] = eval_data.apply(lambda row: tokenize_responses(tokenizer, row["query"], row["response"]), axis=1)

# Drop unecessary columns
train_data = train_data.drop(columns="query").drop(columns="response")
eval_data = eval_data.drop(columns="query").drop(columns="response")

In [ ]:
# Add Lora Matrices
lora_config = LoraConfig(
    r=20,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.4,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Get the PEFT model with LoRA weights
peft_model = get_peft_model(model, lora_config)

In [ ]:
# Setup trainer
training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-4,
    num_train_epochs=5,
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=4000,
    save_steps=1000,
    report_to="wandb"
)

trainer = Trainer(model=peft_model,
    args=training_args,
    train_dataset=QuestionDataset(train_data),
    eval_dataset=QuestionDataset(eval_data)
)

In [ ]:
trainer.train()

In [ ]:
def load_lora_model(
    adapter_name: str,
    model_name: str = "meta-llama/Llama-2-7b-chat-hf",
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name, low_cpu_mem_usage=True, quantization_config=config, attn_implementation="flash_attention_2"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = PeftModel.from_pretrained(model, adapter_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    return model, tokenizer

# Load the model & tokenizer
model, tokenizer = load_lora_model("checkpoints/checkpoint-36000")

In [ ]:
# Load the test data
test_data = pd.read_csv("data/test.csv")

# Tokenize the queries
test_data["input_ids"] = test_data.apply(lambda row: tokenize_queries(tokenizer, row["query"]), axis=1)

# Drop unecessary columns
test_data.drop("query", axis=1, inplace=True)

In [ ]:
correct = 0
total = 0

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in tqdm(range(len(test_data))):
    input_ids = test_data["input_ids"][i].to(device)
    correct_answer = test_data["response"][i][0]

    # Generate the response
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            max_new_tokens=256
        )

    # Decode the response
    outputs = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    
    # Extract the predicted answer
    predicted_answer = outputs[0]

    # Skip if the predicted answer is not a valid answer
    if predicted_answer not in ["A", "B", "C", "D"]:
        continue

    # Check if the predicted answer is correct
    correct += int(predicted_answer == correct_answer)
    total += 1

print(f"Accuracy: {correct}/{total} ({correct / total * 100:.2f}%)")